In [13]:
%%writefile /home/mel_yang/pylib/plateaumodules.py

def mkDary(pD,fh,pops,stable,vary,DorZ,sd="False"):
    ## Make array from large log file containing every combination.
    ## pD=log file directory
    ## fh=filename (including suffix)
    ## pops=list of four populations in order of [P1, P2, P3, P4]
    ## stable=tuple of two indices not varying (must be strings in pops!), second tuple is outgroup!
    ## vary=tuple of two indices that are varying (must be lists in pops!)
    ## DorZ="D" or "Z" indicating what to retrieve for array
    import numpy as np
    
    
    if DorZ=="Z":   
        if sd=="False": DorZind=5 
        else: DorZind=6
    elif DorZ=="D": DorZind=4
    else: return "Not Z or D!"
    
    varypop1=pops[vary[0]]
    varypop2=pops[vary[1]]
    stabpop1=pops[stable[0]]
    stabpop2=pops[stable[1]]
    myary=np.zeros((len(varypop1),len(varypop2)))
    myary[myary==0]=np.nan
    myDs={}
    dfile=open(pD+fh,'r')
    for line in dfile:
        if 'result' not in line: continue
        x=line.split()[1:]
        a,b,c,d=x[:4]
        if stabpop1 not in [a,b,c,d] or stabpop2 not in [a,b,c,d]: continue
        D={}
        D[(a,b,c,d)]=float(x[DorZind])
        D[(b,a,c,d)]=-float(x[DorZind])
        D[(a,b,d,c)]=-float(x[DorZind])
        D[(b,a,d,c)]=float(x[DorZind])
        for dset in D:
            if dset[vary[0]] in varypop1 and dset[vary[1]] in varypop2 and dset[stable[0]]==stabpop1 and dset[stable[1]]==stabpop2:
                p1ind,p2ind=(varypop1.index(dset[vary[0]]),varypop2.index(dset[vary[1]]))
                myary[p1ind,p2ind]=D[dset]
    dfile.close()
    return myary

def mkDary_old(pD,fh,pops,stable,vary,DorZ):
    ## Make array from large log file containing every combination.
    ## pD=log file directory
    ## fh=filename (including suffix)
    ## pops=list of four populations in order of [P1, P2, P3, P4]
    ## stable=tuple of two indices not varying (must be strings in pops!), second tuple is outgroup!
    ## vary=tuple of two indices that are varying (must be lists in pops!)
    ## DorZ="D" or "Z" indicating what to retrieve for array
    
    import numpy as np
    myDs={}
    dfile=open(pD+fh,'r')
    for line in dfile:
        if 'result' not in line: continue
        x=line.split()[1:]
        if x[stable[1]] != pops[stable[1]]: continue
        if x[vary[0]] not in pops[vary[0]]: continue
        if x[vary[1]] not in pops[vary[1]]: continue
        if x[stable[0]] != pops[stable[0]]: continue
        #print x[vary[0]],x[vary[1]],x[stable]
        myDs[(tuple(x[:4]))]=[float(x[4]),float(x[5]),int(x[6]),int(x[7]),int(x[8])]
        revx=[x[1],x[0],x[2],x[3]]
        if x[0]=="Zongri-a" and x[3]=="Mbuti": print (tuple(x[:4])), (tuple(revx))
        myDs[(tuple(revx))]=[-float(x[4]),-float(x[5]),int(x[6]),int(x[7]),int(x[8])]
        
    myary=np.zeros((len(pops[vary[0]]),len(pops[vary[1]])))
    varypop1=pops[vary[0]]
    varypop2=pops[vary[1]]
    for ind1,vp1 in enumerate(varypop1):
        for ind2,vp2 in enumerate(varypop2):
            if len(set([vp1,vp2,pops[stable[0]],pops[stable[1]]]))!=4: myary[ind1,ind2]=np.nan; continue
            mylst=[0,0,0,0]
            mylst[stable[1]]=pops[stable[1]]
            mylst[stable[0]]=pops[stable[0]]
            mylst[vary[0]]=vp1
            mylst[vary[1]]=vp2
            if DorZ=="Z":   DorZind=1 
            elif DorZ=="D": DorZind=0
            else: return "Not Z or D!"
            #myval=myDs[tuple(mylst)][DorZind]
            try:
                myval=myDs[tuple(mylst)][DorZind]
            except KeyError:
                myval=myDs[(mylst[1],mylst[0],mylst[2],mylst[3])][DorZind]
            myary[ind1,ind2]=myval
    return myary

def Zmat2xlsx(myary,newfh,rownames,colnames,popindex1,popindex2):
    import xlsxwriter as xls
    newfile=xls.Workbook(newfh+".xlsx")
    worksheet=newfile.add_worksheet()
    
    mynum = newfile.add_format({'num_format': '0.0','center_across':True})
    bold = newfile.add_format({'bold': True,'align':'center','valign':'vcenter'})
    colheader=newfile.add_format({'bold': True,'align':'center','valign':'vcenter','rotation':90})
    nan = newfile.add_format({'align':'center','valign':'vcenter'})
    
    ##SWITCHED FROM TY FORMATS BC DFREQ DOES OPPOSITE ABBA-BABA (not BABA-ABBA!)
    g2_5 = newfile.add_format({'center_across':True,'bold':True,
                               'font_color':'#9C0006','bg_color':'white',
                               'num_format': '0.0'})
    g3 = newfile.add_format({'center_across':True,'bold':True,'font_color':'#9C0006',
                               'bg_color':'#FFC7CE','num_format': '0.0'})
    l2_5 = newfile.add_format({'center_across':True,'bold':True,'font_color':'#000080',
                               'bg_color':'white','num_format': '0.0'})
    l3 = newfile.add_format({'center_across':True,'bold':True,'font_color':'#000080',
                               'bg_color':'#83CFF6','num_format': '0.0'})
    
    worksheet.set_column(1,len(colnames)+1,4)
    worksheet.set_column(0,0,8)
    row,col=0,0
    worksheet.write_string(row,col,"P%i/P%i" % (popindex1+1,popindex2+1),bold)
    for ind,i in enumerate(colnames): worksheet.write_string(row,col+ind+1,i,colheader)
    for myind in range(myary.shape[0]):
        #print rownames[myind]
        worksheet.write_string(row+myind+1,col,rownames[myind],bold)
        for myind2,i2 in enumerate(myary[myind,:]):
            if str(i2)=="nan": 
                worksheet.write_string(row+myind+1,col+myind2+1,"nan",nan)
            else:
                if i2>=2.5 and i2<2.95: 
                    worksheet.write_number(row+myind+1,col+myind2+1,i2,mynum) #,g2_5)
                elif i2>=2.95:
                    worksheet.write_number(row+myind+1,col+myind2+1,i2,g3)
                elif i2<=-2.5 and i2>-2.95:
                    worksheet.write_number(row+myind+1,col+myind2+1,i2,mynum) #,l2_5)
                elif i2<=-2.95:
                    worksheet.write_number(row+myind+1,col+myind2+1,i2,l3)
                else:
                    worksheet.write_number(row+myind+1,col+myind2+1,i2,mynum)    
    newfile.close()
    return "Made %s!" % newfh


def mkf3ary(pD,fh,pops,stable,vary,DorZ):
    ## Make array from large log file containing every combination.
    ## pD=log file directory
    ## fh=filename (including suffix)
    ## pops=list of three populations in order of [P1, P2, P3]
    ## stable=one index not varying (must be strings in pops!), second tuple is outgroup!
    ## vary=tuple of two indices that are varying (must be lists in pops!)
    ## DorZ="D" or "Z" indicating what to retrieve for array
    
    import numpy as np
    myDs={}
    dfile=open(pD+fh,'r')
    for line in dfile:
        if 'result' not in line: continue
        x=line.split()[1:]

        if x[vary[0]] not in pops[vary[0]]: continue
        if x[vary[1]] not in pops[vary[1]]: continue
        if x[stable] != pops[stable]: continue
        #print x[vary[0]],x[vary[1]],x[stable]
        myDs[(tuple(x[:3]))]=[float(x[3]),float(x[4]),float(x[5]),int(x[6])]

    myary=np.zeros((len(pops[vary[0]]),len(pops[vary[1]])))
    varypop1=pops[vary[0]]
    varypop2=pops[vary[1]]
    for ind1,vp1 in enumerate(varypop1):
        for ind2,vp2 in enumerate(varypop2):
            if len(set([vp1,vp2,pops[stable]]))!=3: myary[ind1,ind2]=np.nan; continue
            mylst=[0,0,0]
            mylst[stable]=pops[stable]
            mylst[vary[0]]=vp1
            mylst[vary[1]]=vp2
            if DorZ=="Z":   DorZind=2 
            elif DorZ=="sd":DorZind=1 
            elif DorZ=="f3":DorZind=0
            else: return "Not Z or f3 or sd!"
            myval=myDs[tuple(mylst)][DorZind]
            myary[ind1,ind2]=myval
    return myary

def piid1(filename,indicestoomit='N'):
    ## PutInfoInDict (PIID) = function to make dictionary automatically from file
    ## puts first line as key, rest as list
    ## if indicestoomit is used, needs to be tuple of numbers, omits those indices: ex. (1,2)
    myfile=open(filename,'r')
    mydict={}
    for line in myfile:
        if line[0]=="#": continue
        x=line.split()
        if indicestoomit != 'N': x=[i for ind,i in enumerate(x) if ind in indicestoomit]
        mydict[x[0]]=x[1:]
    return mydict

def parseoutf3(filename):
    myfile=open(filename,'r')
    myf3dat={}
    for line in myfile:
        x=line.split()
        if x[1] not in myf3dat: myf3dat[x[1]] = {}
        myf3dat[x[1]][(x[1],x[2],x[3])] = [float(i) for i in x[4:-1]] + [int(x[-1])]
    return myf3dat

Overwriting /home/mel_yang/pylib/plateaumodules.py
